Importing libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import urllib.request as urllib2
import re
import csv

Initializing variables

In [25]:
def beautifulSoupWebScrape(urlBase, playerName, loopCount, numPosts, textList_final, repliesList_final, viewsList_final, subforumList_final, topicList_final, dateList_final):
    detectTextCounter = 30 # detecting how many posts in one page
    while detectTextCounter == 30:
        if loopCount == 0: # if it's first page,
            url_final = urlBase+playerName # Then only use the URL base with player name
        else: # else
            url_final = urlBase+playerName+url_add+str(numPosts) # Add the additional URL for more than 30 posts
            numPosts += 30 # Add more 30 in number of posts

#         page = urllib2.urlopen(url_final) # Opening the page
        soupRes = BeautifulSoup(requests.get(url_final).content, 'html.parser') # Web scraping the page
        divcontent_raw = soupRes.find_all("div", class_="content") # Find all <div> tags that contains posts' texts
        dateList = soupRes.find_all("dd", class_="search-result-date") # find all <dd> tags that contains posts' datetime
        ddList = soupRes.find_all("dd") # find all <dd> tags
        detectTextCounter = len(divcontent_raw) # detecting how many posts in current page

        nonNoneCount = 0 # Counter for non-None results
        while nonNoneCount == 0:
            for div in divcontent_raw:
                if div.blockquote != None: # if <blockquote> tags in <div> tags exists
                    nonNoneCount += 1
                    div.blockquote.decompose() # Remove the <blockquote> tags along with the component in it
                if div.span != None: # if <span> tags in <div> tags exists
                    nonNoneCount += 1
                    div.span.unwrap() # Remove the <span> tags
                if div.img != None: # if <img> tags in <div> tags exists
                    nonNoneCount += 1
                    div.img.decompose() # Remove the <img> tags along with the component in it
            for dd in ddList:
                if dd.strong != None: # if <strong> tags in <dd> tags exists
                    nonNoneCount += 1
                    dd.strong.unwrap() # Remove the <strong> tags
                if dd.a != None: # if <a> tags in <d> tags exists
                    nonNoneCount += 1
                    dd.a.unwrap() # Remove the <a> tags along with the component in it
            if nonNoneCount != 0:
                nonNoneCount = 0
            else:
                break

        repliesList_raw = [] # list of posts' replies number for current page
        viewsList_raw = [] # list of posts' views number for current page
        subforumList_raw = [] # list of posts' subforum for current page
        topicList_raw = [] # list of posts' topic for current page
        for i in range(len(ddList)):
            if i == 0:
                continue
            dd_text = ddList[i].get_text()
            if "Replies:" in dd_text: # if 'Replies' word is part of the text
                repliesList_raw.append(dd_text) # add the replies number to the list for current page
            if "Views:" in dd_text: # if 'Views' word is part of the text
                viewsList_raw.append(dd_text) # add the views number to the list for current page
            if "Forum:" in dd_text: # if 'Forum' word is part of the text
                subforumList_raw.append(dd_text) # add the subforum title to the list for current page
            if "Topic:" in dd_text: # if 'Topic' word is part of the text
                topicList_raw.append(dd_text) # add the topic title to the list for current page

        textList_raw = []
        for div in divcontent_raw:
            # Removing every unnecessary parts of the posts' texts (such as enter, long whitespaces, 
            # leading and tailing whitespaces and '...')
            textList_raw.append(div.get_text().replace("\n", "").replace("    ", " ").replace("...", "").replace("  ", " ").replace("   ", " ").strip())
            
#         print(len(textList_raw), len(repliesList_raw), len(viewsList_raw), len(subforumList_raw), len(topicList_raw), len(dateList))

        for i in range(detectTextCounter):
            result = re.sub(r'http\S+', '', textList_raw[i], flags=re.MULTILINE) # Remove every URL in the posts' texts
            textList_final.append(result) # Append the posts' texts in the final list
            
            # Append the posts' replies number in the final list
            repliesList_final.append(int(repliesList_raw[i].split(": ")[1]))
            
            # Append the posts' views number in the final list
            viewsList_final.append(int(viewsList_raw[i].split(": ")[1]))
            
            # Append the posts' replies number in the final list
            subforumList_final.append(subforumList_raw[i].split(": ")[1])
            
            # Append the posts' views number in the final list
            topicList_final.append(topicList_raw[i].split(": ")[1])
            
            # Append the posts' datetime in the final list
            dateList_final.append(dateList[i].get_text())

        print(url_final) # print the URL destination
        loopCount += 1 # adding counter for number of page scraped

In [1]:
# URL
url_base = "http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=" # URL base
playerNames = ['Ronaldo', 'Messi', 'Pele', 'Maradona', 'Rooney']
url_add = "&start=" # URL additional for more than 30 posts

# for loop parts
loopCount_list = []
for i in range(len(playerNames)):
    loopCount_list.append(0)
numPosts_list = []
for i in range(len(playerNames)):
    numPosts_list.append(30)
loopCount_test = 0 # Loop counter for scanning Ronaldo-related posts
numPosts_test = 30 # Number of Ronaldo-related posts

# Lists
# textList_final_test = []
textlist_final_list = []
for i in range(len(playerNames)):
    textlist_final_list.append(list())
# repliesList_final_test = []
repliesList_final_list = []
for i in range(len(playerNames)):
    repliesList_final_list.append(list())
# viewsList_final_test = []
viewsList_final_list = []
for i in range(len(playerNames)):
    viewsList_final_list.append(list())
# subforumList_final_test = []
subforumList_final_list = []
for i in range(len(playerNames)):
    subforumList_final_list.append(list())
# viewsList_final_test = []
topicList_final_list = []
for i in range(len(playerNames)):
    topicList_final_list.append(list())
dateList_final_test = []
dateList_final_list = []
for i in range(len(playerNames)):
    dateList_final_list.append(list())

In [27]:
for i in range(len(playerNames)):
    beautifulSoupWebScrape(url_base, playerNames[i], loopCount_list[i], numPosts_list[i], textlist_final_list[i], repliesList_final_list[i], viewsList_final_list[i], subforumList_final_list[i], topicList_final_list[i], dateList_final_list[i])

http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=30
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=60
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=90
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=120
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=150
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=180
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=210
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=240
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=270
http://www.footballnews

http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2490
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2520
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2550
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2580
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2610
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2640
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2670
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2700
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2730
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=2760


http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=4950
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=4980
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5010
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5040
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5070
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5100
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5130
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5160
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5190
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=5220


http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7410
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7440
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7470
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7500
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7530
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7560
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7590
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7620
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7650
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Ronaldo&start=7680


http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=90
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=120
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=150
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=180
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=210
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=240
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=270
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=300
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=330
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=360
http://www.footballnews.com.au/

http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2610
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2640
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2670
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2700
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2730
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2760
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2790
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2820
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2850
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Messi&start=2880
http://www.footballn

http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=240
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=270
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=300
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=330
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=360
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=390
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=420
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=450
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=480
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Maradona&start=510


http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=480
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=510
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=540
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=570
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=600
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=630
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=660
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=690
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=720
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=750
http://www.footballn

http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=2970
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3000
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3030
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3060
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3090
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3120
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3150
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3180
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3210
http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=3240
http://www

http://www.footballnews.com.au/forum/search.php?st=0&sk=t&sd=d&sr=posts&keywords=Rooney&start=5460


In [28]:
textList_final_noDup_list = [] # List of posts' text with no duplicates
repliesList_final_nodup_list = [] # List of posts' replies number with no duplicates
viewsList_final_noDup_list = [] # List of posts' views number with no duplicates
subforumList_final_noDup_list = [] # List of posts' subforum titles with no duplicates
topicList_final_noDup_list = [] # List of posts' topic titles with no duplicate
dateList_final_noDup_list = [] # List of posts' datetime with no duplicate
for i in range(len(playerNames)):
    textList_final_noDup_list.append(list())
    repliesList_final_nodup_list.append(list())
    viewsList_final_noDup_list.append(list())
    subforumList_final_noDup_list.append(list())
    topicList_final_noDup_list.append(list())
    dateList_final_noDup_list.append(list())
    
for i in range(len(playerNames)):
    for j in range(len(textlist_final_list[i])):
        if textlist_final_list[i][j] not in textList_final_noDup_list[i]: # if text does not exist in the no-duplicate list
            # Added to the posts' texts' no-duplicate list
            textList_final_noDup_list[i].append(textlist_final_list[i][j])
            # Added to the posts' replies numbers' no-duplicate list
            repliesList_final_nodup_list[i].append(repliesList_final_list[i][j])
            # Added to the posts' views numbers' no-duplicate list
            viewsList_final_noDup_list[i].append(viewsList_final_list[i][j])
            # Added to the posts' subforum titles' no-duplicate list
            subforumList_final_noDup_list[i].append(subforumList_final_list[i][j])
            # Added to the posts' topic titles' no-duplicate list
            topicList_final_noDup_list[i].append(topicList_final_list[i][j])
            # Added to the posts' datetime' no-duplicate list
            dateList_final_noDup_list[i].append(dateList_final_list[i][j])

In [29]:
for i in range(len(playerNames)):
    print(len(textlist_final_list[i]), len(textList_final_noDup_list[i]))
# print(textList_final_noDup_list[0])

9808 8021
3561 2977
1342 1071
2190 1754
5460 4365


In [30]:
filename_head = "data_"
filename_tail = "_rep_view_final.csv"
for i in range(len(playerNames)):
    filename_final = filename_head+playerNames[i].lower()+filename_tail
    with open(filename_final, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for j in range(len(textList_final_noDup_list[i])):
#             print(dateList_final_noDup_list[i][j])
            writer.writerow([textList_final_noDup_list[i][j], dateList_final_noDup_list[i][j], repliesList_final_nodup_list[i][j], viewsList_final_noDup_list[i][j], subforumList_final_noDup_list[i][j], topicList_final_noDup_list[i][j]])